In [14]:
from topologicpy.Vertex import Vertex
from topologicpy.Edge import Edge 
from topologicpy.Wire import Wire 
from topologicpy.Face import Face 
from topologicpy.Shell import Shell
from topologicpy.Cell import Cell
from topologicpy.CellComplex import CellComplex
from topologicpy.Cluster import Cluster
from topologicpy.Topology import Topology
from topologicpy.Dictionary import Dictionary
from topologicpy.Graph import Graph 

In [15]:
d1 = CellComplex.Prism(origin=(0,0,0),uSides=1, vSides=1, wSides=10)
d2 = CellComplex.Prism(origin=(0,3,0),uSides=1, vSides=1, wSides=10)
d3 = CellComplex.Prism(origin=(3,0,0),uSides=1, vSides=1, wSides=10)
d4 = CellComplex.Prism(origin=(3,3,0),uSides=1, vSides=1, wSides=10)

#Topology.Show(d1,d2,d3,d4,renderer="browser")

In [16]:
g_width = 20
g_length= 10
g_height = 1
g_sides = 3

num_floors = 3

c_width = 0.5
c_length = 0.5
c_height = 3

ground = CellComplex.Prism(width=g_width, length=g_length, height= g_height, uSides =g_sides,wSides=1, vSides=g_sides,placement= "bottom")
cells = Topology.Cells(ground)

centroids = []
columns = []

for cell in cells:
    centroid = Topology.Centroid(cell)
    centroid = Topology.Translate(centroid,0,0,g_height*0.5)
    centroids.append(centroid)
    column = Cell.Prism(origin = centroid,width=c_width, length = c_length, height= c_height, placement= "bottom")
    columns.append(column)

centroid = Topology.Centroid(ground)
centroid = Topology.Translate(centroid,0,0,g_height*0.5+c_height)

b_width = g_width*0.9
b_length = g_length*0.9 
b_u_sides = int(b_width/3)
b_v_sides = int(b_length/3)

block = CellComplex.Prism(width=g_width*0.9, 
                          length=g_length*0.9,
                          height=num_floors*3,
                          uSides=b_u_sides,
                          vSides=b_v_sides,
                          wSides=num_floors,
                          placement="bottom")

Topology.Show(ground,columns,block,centroids,xAxis=True, yAxis=True, zAxis=True,renderer="browser")

In [22]:
#final_result = Topology.SelfMerge(Cluster.ByTopologies(ground,column,block))
#print(final_result)

ground_cells = Topology.Cells(ground)

selectors = []
for ground_cell in ground_cells:
    centroid = Topology.Centroid(ground_cell)
    #ground cell are going to be labelled as 0 
    dc = Dictionary.ByKeyValue("id",0)
    centroid = Topology.SetDictionary(centroid,dc)
    selectors.append(centroid)

for column in columns:
    centroid = Topology.Centroid(column)
    #column are going to be labelled as 1 
    dc = Dictionary.ByKeyValue("id",1)
    centroid = Topology.SetDictionary(centroid,dc)
    selectors.append(centroid)


block_cells = Topology.Cells(block)
for block_cell in block_cells:
    centroid = Topology.Centroid(block_cell)
    #block cell are going to be labelled as 2 
    dc = Dictionary.ByKeyValue("id",2)
    centroid = Topology.SetDictionary(centroid,dc)
    selectors.append(centroid)


all_cells = ground_cells+block_cells+ columns
final_result = CellComplex.ByCells(all_cells)

final_result = Topology.TransferDictionariesBySelectors(final_result,selectors,tranCells=True)
print(final_result)

In [24]:
g = Graph.ByTopology(final_result)
#Topology.Show(g, final_result,faceOpacity=0.2, renderer="browser")

Graph.Show(g,vertexSize=6, vertexGroupKey="id", vertexGroups=[0,1,2], renderer="browser")